In [1]:
import pickle
with open("data2.pickle","rb") as fr:
    data = pickle.load(fr)
data

,접수월,가맹점업종분류명_대,가맹점업종분류명,연령대코드,성별코드,광역시도명,시군구명,금액지표,건수지표,금액등급,건수등급
0,202207,보건/위생,화 장 품,70,1,강원,강릉시,145.0,89.0,2,3
1,202207,보건/위생,의료 용품,30,1,강원,강릉시,56.0,157.0,4,2
2,202207,보건/위생,의료 용품,40,1,강원,강릉시,800.0,290.0,1,1
3,202207,보건/위생,사우나,30,2,강원,강릉시,44.0,27.0,4,4
4,202207,보건/위생,사우나,60,2,강원,강릉시,45.0,39.0,4,4
...,...,...,...,...,...,...,...,...,...,...,...
3548444,202306,기타,기계 공구,50,2,충북,충주시,71.0,127.0,3,2
3548445,202306,기타,기계 공구,60,2,충북,충주시,174.0,142.0,2,2
3548446,202306,기타,비 영 리,50,1,충북,충주시,88.0,142.0,3,2
3548447,202306,기타,비 영 리,70,2,충북,충주시,51.0,40.0,4,4


In [2]:
# 파생feature생성
data['최종등급'] = data.금액등급.map(str) \
                            + data.건수등급.map(str)
data['지역'] = data.광역시도명.map(str) \
                            + data.시군구명.map(str)

In [5]:
# 최종등급 하나의 문자로 대치
data = data.replace({'최종등급' : '11'}, 'A')
data = data.replace({'최종등급' : '12'}, 'B')
data = data.replace({'최종등급' : '13'}, 'C')
data = data.replace({'최종등급' : '14'}, 'D')
data = data.replace({'최종등급' : '21'}, 'E')
data = data.replace({'최종등급' : '22'}, 'F')
data = data.replace({'최종등급' : '23'}, 'G')
data = data.replace({'최종등급' : '24'}, 'H')
data = data.replace({'최종등급' : '31'}, 'I')
data = data.replace({'최종등급' : '32'}, 'J')
data = data.replace({'최종등급' : '33'}, 'K')
data = data.replace({'최종등급' : '34'}, 'L')
data = data.replace({'최종등급' : '41'}, 'M')
data = data.replace({'최종등급' : '42'}, 'N')
data = data.replace({'최종등급' : '43'}, 'O')
data = data.replace({'최종등급' : '44'}, 'P')

In [6]:
from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()

data['가맹점업종분류명_대']=encoder.fit_transform(data['가맹점업종분류명_대'])
data['가맹점업종분류명']=encoder.fit_transform(data['가맹점업종분류명'])
data['연령대코드']=encoder.fit_transform(data['연령대코드'])
data['성별코드']=encoder.fit_transform(data['성별코드'])
data['광역시도명']=encoder.fit_transform(data['광역시도명'])
data['시군구명']=encoder.fit_transform(data['시군구명'])

In [7]:
# feature제거
model_df = data.drop('지역', axis = 1)
model_df = model_df.drop('접수월', axis = 1)

In [9]:
model_df['최종등급']=encoder.fit_transform(model_df['최종등급'])

In [10]:
# feature제거
model_df = model_df.drop('금액등급', axis = 1)
model_df = model_df.drop('건수등급', axis = 1)

model_df = model_df.drop('금액지표', axis = 1)
model_df = model_df.drop('건수지표', axis = 1)

In [12]:
model_df #최종data

,가맹점업종분류명_대,가맹점업종분류명,연령대코드,성별코드,광역시도명,시군구명,최종등급
0,12,252,6,0,0,3,6
1,12,160,2,0,0,3,13
2,12,160,3,0,0,3,0
3,12,110,2,1,0,3,15
4,12,110,5,1,0,3,15
...,...,...,...,...,...,...,...
3548444,5,31,4,1,17,208,9
3548445,5,31,5,1,17,208,5
3548446,5,103,4,0,17,208,9
3548447,5,103,6,1,17,208,15


### model1. RandomForest

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, mean_squared_error,mean_absolute_error,mean_absolute_percentage_error,mean_squared_error,mean_squared_log_error,r2_score

X = model_df.drop(['최종등급'], axis = 1)
y = model_df['최종등급']

# feature scaling
scaler = StandardScaler( )
X_scaled = scaler.fit_transform(X)

#data split
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(X_scaled, y, test_size = 0.2, random_state = 156, stratify=y)

In [16]:
#randomfores(1)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
rf_clf.fit(X_train_scaled, y_train_scaled)
pred_rf = rf_clf.predict(X_test_scaled)

In [17]:
from sklearn.metrics import accuracy_score, confusion_matrix

print("정확도 : {:.3f}".format(accuracy_score(y_test_scaled,pred_rf)))
print("오차 행렬 \n",confusion_matrix(y_test_scaled,pred_rf))

정확도 : 0.587
오차 행렬 
 [[113216   4777    110     10   5279   5180    248     24    553    629
     302     69    307    224    178    422]
 [  7100  13288    993     50    386   8169   1905    104     52    758
     837    183     32    240    308    355]
 [   218   1648   1579    283     17    544   1515    332      4    101
     616    281      2     43    253    378]
 [    42    114    354    663      2     36    173    409      0      6
      78    285      2      7     46    527]
 [  8334    396     13      2  10892   3361     55      8   1528   1452
     139     22    565    429    184    153]
 [  5907   5731    238     17   2583  58593   5361     74    160   5681
    7110    372     56    715   1127   1261]
 [   511   2159    976     77     65   7471  16163    643      5    589
   10416   2217      5     92   1001   2260]
 [    57    216    325    323      6    159   1135   1544      0     30
     529   2092      0     17    181   2136]
 [  1148     48      2      0   2158    237 

In [18]:
from sklearn.metrics import classification_report

print(classification_report(y_test_scaled,pred_rf))

              precision    recall  f1-score   support

           0       0.81      0.86      0.84    131528
           1       0.43      0.38      0.40     34760
           2       0.28      0.20      0.23      7814
           3       0.35      0.24      0.29      2744
           4       0.45      0.40      0.42     27533
           5       0.56      0.62      0.59     94986
           6       0.41      0.36      0.38     44650
           7       0.27      0.18      0.21      8750
           8       0.34      0.27      0.30      8390
           9       0.37      0.33      0.35     33377
          10       0.54      0.57      0.55     93001
          11       0.45      0.37      0.41     43911
          12       0.56      0.54      0.55      6749
          13       0.39      0.36      0.38     13773
          14       0.38      0.35      0.37     33526
          15       0.71      0.79      0.75    124198

    accuracy                           0.59    709690
   macro avg       0.46   

In [19]:
# 각 feature 중요도
rf_clf.feature_importances_

array([0.10826682, 0.20318124, 0.10124757, 0.0218573 , 0.10230543,
       0.46314164])

In [20]:
#randomforest(2)
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=100, random_state=0, max_depth=24, min_samples_split=2)
rf_clf.fit(X_train_scaled, y_train_scaled)
pred_rf = rf_clf.predict(X_test_scaled)

In [21]:
print("정확도 : {:.3f}".format(accuracy_score(y_test_scaled,pred_rf)))
print("오차 행렬 \n",confusion_matrix(y_test_scaled,pred_rf))

정확도 : 0.564
오차 행렬 
 [[114915   3198     45      1   3276   5798    190     10    290    391
     587     58    211    132    135   2291]
 [  9053  10097    551     16    202   8879   1496     50     24    485
    1263    126     26    124    259   2109]
 [   710   1313    912    148      8    624   1212    155      0     70
     789    164      6     18    208   1477]
 [   182    101    211    414      2     69    114    222      0      7
     101    163      0      3     38   1117]
 [ 10876    260      9      1   8081   3909     50      1    935   1043
     320     15    419    247    132   1235]
 [  8464   3872    105      4   1637  59095   3755     26     78   3684
    8333    232     43    414    837   4407]
 [  1576   1555    483     27     35   8225  12404    293      1    341
   12027   1515      4     44    713   5407]
 [   403    163    165    161      2    251    871    853      0     24
     610   1352      2      9    121   3763]
 [  2166     33      1      1   1662    333 

In [22]:
from sklearn.metrics import classification_report

print(classification_report(y_test_scaled,pred_rf))

              precision    recall  f1-score   support

           0       0.72      0.87      0.79    131528
           1       0.45      0.29      0.35     34760
           2       0.31      0.12      0.17      7814
           3       0.44      0.15      0.22      2744
           4       0.48      0.29      0.36     27533
           5       0.53      0.62      0.57     94986
           6       0.43      0.28      0.34     44650
           7       0.32      0.10      0.15      8750
           8       0.38      0.19      0.25      8390
           9       0.39      0.24      0.30     33377
          10       0.51      0.57      0.54     93001
          11       0.49      0.27      0.35     43911
          12       0.59      0.46      0.52      6749
          13       0.43      0.26      0.33     13773
          14       0.39      0.25      0.30     33526
          15       0.59      0.83      0.69    124198

    accuracy                           0.56    709690
   macro avg       0.47   

In [23]:
rf_clf.feature_importances_

array([0.14284958, 0.2472034 , 0.13290442, 0.02483263, 0.10169507,
       0.3505149 ])

### smote 적용해보기
##### 데이터 불균형 해결

In [26]:
#pip install imblearn
import imblearn

print(imblearn.__version__)

0.11.0


In [27]:
y_train_scaled.value_counts()

0     526110
15    496789
5     379945
10    372004
6     178599
11    175646
1     139042
14    134106
9     133508
4     110134
13     55093
7      34999
8      33558
2      31254
12     26994
3      10978
Name: 최종등급, dtype: int64

In [29]:
# smote적용 전후 비교
from imblearn.over_sampling import SMOTE
import pandas as pd

smote = SMOTE(random_state=0)
X_train_over, y_train_over = smote.fit_resample(X_train_scaled, y_train_scaled)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_train_scaled.shape, y_train_scaled.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (2838759, 6) (2838759,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (8417760, 6) (8417760,)
SMOTE 적용 후 레이블 값 분포: 
 4     526110
15    526110
10    526110
9     526110
14    526110
3     526110
0     526110
6     526110
5     526110
8     526110
11    526110
1     526110
7     526110
12    526110
2     526110
13    526110
Name: 최종등급, dtype: int64


In [31]:
#smote 적용 후 randomforest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

rf_clf_sm = RandomForestClassifier(n_estimators=100, random_state=0)
rf_clf.fit(X_train_over, y_train_over)
pred_rf = rf_clf.predict(X_test_scaled)

In [32]:
from sklearn.metrics import accuracy_score, confusion_matrix

print("정확도 : {:.3f}".format(accuracy_score(y_test_scaled,pred_rf)))
print("오차 행렬 \n",confusion_matrix(y_test_scaled,pred_rf))

정확도 : 0.508
오차 행렬 
 [[95197  8273   914   551  9665  4784   790   671  1938  1677   872   527
   1448  1382  1131  1708]
 [ 4540 14096  2318   446   837  4815  1908   567   290  1190   623   366
    248   767   983   766]
 [  208  1291  2252   708    56   195   913   543    57   153   211   172
     77   198   457   323]
 [   41    95   419  1044    10    15    63   481    13    25    23    98
     43    34    98   242]
 [ 4748   791   196   133 11937  1982   139   166  2694  1677   221   140
   1064   769   448   428]
 [ 4424  8427  1182   395  4655 44407  7608   695   608  8537  5870  1186
    366  1704  2564  2358]
 [  752  2740  2081   536   288  4813 15871  1787   162  1119  6724  2696
    179   518  2146  2238]
 [  109   257   617   868    57    81   726  2560    50    92   230  1284
     70   121   424  1204]
 [  596   108    59    63  1673    78    24    56  2908   427    21    20
   1331   726   164   136]
 [  981  1303   492   267  2049  4373   811   325   982 11463  2427   2

In [33]:
from sklearn.metrics import classification_report

print(classification_report(y_test_scaled,pred_rf))

              precision    recall  f1-score   support

           0       0.83      0.72      0.77    131528
           1       0.34      0.41      0.37     34760
           2       0.15      0.29      0.20      7814
           3       0.11      0.38      0.17      2744
           4       0.35      0.43      0.39     27533
           5       0.60      0.47      0.52     94986
           6       0.35      0.36      0.35     44650
           7       0.14      0.29      0.19      8750
           8       0.23      0.35      0.28      8390
           9       0.30      0.34      0.32     33377
          10       0.56      0.43      0.48     93001
          11       0.38      0.39      0.39     43911
          12       0.35      0.60      0.44      6749
          13       0.26      0.39      0.31     13773
          14       0.28      0.39      0.33     33526
          15       0.70      0.64      0.67    124198

    accuracy                           0.51    709690
   macro avg       0.37   